# <a name="0">Machine Learning Accelerator - Natural Language Processing - Lecture 2</a>

## Recurrent Neural Networks (RNNs) for the Product Review Problem - Classify Product Reviews as Positive or Not

In this exercise, we will learn how to use Recurrent Neural Networks. 

We will follow these steps:
1. <a href="#1">Reading the dataset</a>
2. <a href="#2">Exploratory data analysis</a>
3. <a href="#3">Train-validation dataset split</a>
4. <a href="#4">Text Transformation</a>
5. <a href="#5">Generating data batch and iterator</a>
6. <a href="#6">Using pre-trained GloVe Word Embeddings</a>
7. <a href="#7">Setting Hyperparameters and Bulding the Network</a>
8. <a href="#8">Training the Network</a>
9. <a href="#9">Improvement ideas</a>

Overall dataset schema:
* __reviewText:__ Text of the review
* __summary:__ Summary of the review
* __verified:__ Whether the purchase was verified (True or False)
* __time:__ UNIX timestamp for the review
* __log_votes:__ Logarithm-adjusted votes log(1+votes)
* __isPositive:__ Whether the review is positive or negative (1 or 0)

__Important note:__ One big distinction betweeen the regular neural networks and RNNs is that RNNs work with sequential data. In our case, RNNs will help us with the text field. If we also want to consider other fields such as time, log_votes, verified, etc., we need to use the regular neural networks with the RNN network.

In [1]:
# Upgrade dependencies
!pip install -r ../../requirements.txt

In [2]:
import re, time
import numpy as np
import torch, torchtext
import boto3
import os
import pandas as pd

from os import path
from collections import Counter
from torch import nn, optim
from torch.nn import BCEWithLogitsLoss
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from torchtext.vocab import GloVe

## 1. <a name="1">Reading the dataset</a>
(<a href="#0">Go to top</a>)

Let's read the dataset below and look at the first five rows in the dataset. 

In [3]:
df = pd.read_csv("../../data/examples/NLP-REVIEW-DATA-CLASSIFICATION-TRAINING.csv")
df.head()

,ID,reviewText,summary,verified,time,log_votes,isPositive
0,65886,Purchased as a quick fix for a needed Server 2...,"Easy install, seamless migration",True,1458864000,0.000000,1.0
1,19822,So far so good. Installation was simple. And r...,Five Stars,True,1417478400,0.000000,1.0
2,14558,Microsoft keeps making Visual Studio better. I...,This is the best development tool I've ever used.,False,1252886400,0.000000,1.0
3,39708,Very good product.,Very good product.,True,1458604800,0.000000,1.0
4,8015,So very different from my last version and I a...,... from my last version and I am having a gre...,True,1454716800,2.197225,0.0


## 2. <a name="2">Exploratory data analysis</a>
(<a href="#0">Go to top</a>)

Let's look at the range and distribution of the target column `isPositive`.

In [4]:
df["isPositive"].value_counts()

1.0    34954
0.0    21046
Name: isPositive, dtype: int64

We can check the number of missing values for each columm below.

In [5]:
print(df.isna().sum())

ID             0
reviewText    10
summary       12
verified       0
time           0
log_votes      0
isPositive     0
dtype: int64


We have missing values in our text fields. We will use the __reviewText__ field, so we fill-in the missing values in it iwth the empty string.

In [6]:
df["reviewText"] = df["reviewText"].fillna("missing")

## 3. <a name="3">Train-validation split</a>
(<a href="#0">Go to top</a>)

Let's split the dataset into training and validation.

In [7]:
# This separates 10% of the entire dataset into validation dataset.
train_text, val_text, train_label, val_label = train_test_split(
    df["reviewText"].tolist(),
    df["isPositive"].tolist(),
    test_size=0.10,
    shuffle=True,
    random_state=324,
)

## 4. <a name="4">Text transformation</a>
(<a href="#0">Go to top</a>)

We will apply the following processes here:
1. Creating a vocabulary
2. Text transformation

__1. Creating a vocabulary:__ 

We will create a vocabulary with the tokens from the text data. We use a simple english tokenizer and use these tokens to create our vocabulary. In this vocabulary, tokens will map to unique ids, such as "car"->32, "house"->651, etc. 

In [8]:
tokenizer = get_tokenizer("basic_english")
counter = Counter()
for line in train_text:
    counter.update(tokenizer(line))
vocab = Vocab(counter, min_freq=2) #min_freq>1 for skipping misspelled words

Here are some examples.

In [9]:
print(f"'home' -> {vocab['home']}")
print(f"'wash' -> {vocab['wash']}")
# unknown word (assume from test set)
print(f"'fhshbasdhb' -> {vocab['fhshbasdhb']}")

'home' -> 211
'wash' -> 10241
'fhshbasdhb' -> 0


Let's print the words for the first 25 indexes in the vocabulary. '< unk >' is reserved for unknown words '< pad >' is used for the padded tokens (we will learn in a minute).

In [10]:
print(vocab.itos[0:25])

['<unk>', '<pad>', '.', 'the', 'i', ',', 'to', 'and', 'it', 'a', "'", 'of', 'is', 'for', 'this', 'you', 'that', 'my', 'with', 'in', 'have', 'on', 'not', 'was', 'but']


__2. Text transformation:__ 

We will use the vocabulary and map tokens in the text to unique ids of the tokens. For example: `["this", "is", "a", "sentence"] -> [14, 12, 9, 2066]`.

In [11]:
# Let's create a mapper to transform our text data
text_transform_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]

Let's see some text before and after transformation.

In [12]:
print(f"Before transform:\t{train_text[37]}")
print(f"After transform:\t{text_transform_pipeline(train_text[37])}")

Before transform:	Happy to own it.
After transform:	[321, 6, 237, 8, 2]


Let's create a function for this. In this function, we transform and pad (if necessary) our text data. We cut the series of words at the point where it reaches a certain lenght (we used `max_len=50` here). If the text is shorter than max_len, we `pad ones` to the start of the sequence.

In [13]:
def pad_features(reviews_split, seq_length):
    # Transform the text
    # use the dict to tokenize each review in reviews_split
    # store the tokenized reviews in reviews_ints
    reviews_ints = []
    for review in reviews_split:
        reviews_ints.append(text_transform_pipeline(review))
    
    # getting the correct rows x cols shape
    features = np.ones((len(reviews_ints), seq_length), dtype=int)
    
    # for each review, I grab that review
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return torch.tensor(features, dtype=torch.int64)

Let's see two example sentences. Remember that 1 is used for each padded item and 0 is used for each unknown word (if there is any in the given text).

In [14]:
for text in train_text[9:11]:
    print(f"Text: {text}\n")
    print(f"Original length of the text: {len(text)}\n")
    tt = pad_features([text], seq_length=50)
    print(f"Transformed text: \n{tt}\n")
    print(f"Shape of transformed text: {tt.shape}\n")

Text: Didnt give a 5 because I don't know what I need. I like it great

Original length of the text: 64

Transformed text: 
tensor([[   1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1, 4934,  240,    9,  189,
          100,    4,   85,   10,   25,  155,   67,    4,  109,    2,    4,   60,
            8,   68]])

Shape of transformed text: torch.Size([1, 50])

Text: I downloaded the 30 day trial onto my wife's Mac about a week ago, but I don't know if I did it correctly as she is still getting spam..but not quite as much. Perhaps someone has information on this as I don't find a way to send an email to the company to see if I am in the trial period.

Original length of the text: 288

Transformed text: 
tensor([[   4,  347,    3,  695,  307,  559,  895,   17,  990,   10,   29,  132,
           83,    9,  598,  354,    5,   24,    4,

## 5. <a name="5">Generating data batch and iterator</a>
(<a href="#0">Go to top</a>)

Let's use the pad_features() function and create the data loaders. Here, we use __max_len=50__ to consider the first 50 words in the text.

In [15]:
max_len = 50
batch_size = 64

# Pass transformed and padded data to dataset
# Create data loaders
train_dataset = TensorDataset(
    pad_features(train_text, max_len), torch.tensor(train_label, dtype=torch.float32)
)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

val_dataset = TensorDataset(pad_features(val_text, max_len), torch.tensor(val_label, dtype=torch.float32))
val_loader = DataLoader(val_dataset, batch_size=batch_size)

## 6. <a name="6">Using pre-trained GloVe word embeddings</a>
(<a href="#0">Go to top</a>)

In this example, we will use GloVe word vectors. `name='6B'` `dim=300` gives us 6 billion words/phrases vectors. Each word vector has 300 numbers in it. The following code shows how to get the word vectors and create an embedding matrix from them. We will connect our vocabulary indexes to the GloVe embedding with the `get_vecs_by_tokens()` function.

In [16]:
glove = GloVe(name="6B", dim=300)
embedding_matrix = glove.get_vecs_by_tokens(vocab.itos)

## 7. <a name="7">Setting hyperparameters and bulding the network</a>
(<a href="#0">Go to top</a>)

We will set our parameters like below.

In [17]:
# Size of the state vectors
hidden_size = 128

# General NN training parameters
learning_rate = 0.0001
epochs = 15

# Embedding vector and vocabulary sizes
embed_size = 300  # glove.6B.300d.txt
vocab_size = len(vocab.itos)

We need to put our data into correct format before the process.

Our model is made of these layers:
* Embedding layer: This is where our words/tokens are mapped to word vectors.
* RNN layer: We are using a simple RNN model. We stack 2 RNN layers in this example. More details about the RNN are available [here](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html).
* Linear layer: A linear layer with a single neuron is used to output the `isPositive` prediction.

In [18]:
class Net(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=1)
        self.rnn = nn.RNN(
            embed_size, hidden_size, num_layers=num_layers, batch_first=True
        )

        self.linear = nn.Linear(hidden_size, 1)
        self.act = nn.Sigmoid()

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        # Call the RNN layer
        outputs, _ = self.rnn(embeddings)
        
        # Output shape after RNN: (batch_size, max_len, hidden_size)
        # Get the output from the last time step with outputs[:, -1, :] below
        # The output shape becomes: (batch_size, 1, hidden_size)
        # Send it to the linear layer
        outs = self.linear(outputs[:, -1, :])
        return self.act(outs)
    
# Initialize the weights
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
    if type(m) == nn.RNN:
        for param in m._flat_weights_names:
            if "weight" in param:
                nn.init.xavier_uniform_(m._parameters[param])

Let's initialize this network. Then, we will need to make the embedding layer use our GloVe word vectors.

In [19]:
# Our architecture with 2 RNN layers
model = Net(vocab_size, embed_size, hidden_size, num_layers=2)

# We set the embedding layer's parameters from GloVe
model.embedding.weight.data.copy_(embedding_matrix)
# We won't change/train the embedding layer
model.embedding.weight.requires_grad = False

## 8. <a name="8">Training the network</a>
(<a href="#0">Go to top</a>)

Now, it is time to start our training. We define the loss function and training algorithm first. Then, training starts!

We will define the trainer and loss function below. 

__Binary cross-entropy loss__ is used as this is a binary classification problem.

$$
\mathrm{BinaryCrossEntropyLoss} = -\sum_{examples}{(y\log(p) + (1 - y)\log(1 - p))}
$$

In [20]:
# Setting our trainer
trainer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# We will use Binary Cross-entropy loss
# reduction="sum" sums the losses for given output and target
cross_ent_loss = nn.BCELoss(reduction="sum")

Now, it is time to start the training process. We will print the Binary cross-entropy loss loss after each epoch.

In [21]:
# Get the compute device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.apply(init_weights)
model.to(device)

for epoch in range(epochs):
    start = time.time()
    training_loss = 0
    val_loss = 0
    # Training loop, train the network
    for data, target in train_loader:
        trainer.zero_grad()
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        L = cross_ent_loss(output, target.unsqueeze(1))
        training_loss += L.item()
        L.backward()
        trainer.step()

    # Validate the network, no training (no weight update)
    for data, target in val_loader:
        val_predictions = model(data.to(device))
        L = cross_ent_loss(val_predictions, target.to(device).unsqueeze(1))
        val_loss += L.item()

    # Let's take the average losses
    training_loss = training_loss / len(train_label)
    val_loss = val_loss / len(val_label)

    end = time.time()
    print(
        f"Epoch {epoch}. Train_loss {training_loss}. Val_loss {val_loss}. Seconds {end-start}"
    )

Epoch 0. Train_loss 0.6261530711348094. Val_loss 0.570370820590428. Seconds 20.90886092185974
Epoch 1. Train_loss 0.5600190313278682. Val_loss 0.5283386766910553. Seconds 21.350273847579956
Epoch 2. Train_loss 0.5333711538806795. Val_loss 0.5153359385899136. Seconds 20.032791137695312
Epoch 3. Train_loss 0.5150970947174799. Val_loss 0.502014913218362. Seconds 20.28631567955017
Epoch 4. Train_loss 0.5012848254990956. Val_loss 0.49316429955618724. Seconds 20.181519746780396
Epoch 5. Train_loss 0.48963918178800553. Val_loss 0.4873343655041286. Seconds 20.40234661102295
Epoch 6. Train_loss 0.480525227009304. Val_loss 0.48011997972215925. Seconds 20.62388277053833
Epoch 7. Train_loss 0.47264597958988613. Val_loss 0.47484635250908985. Seconds 20.39682173728943
Epoch 8. Train_loss 0.46557222362548584. Val_loss 0.4719020448412214. Seconds 20.268911361694336
Epoch 9. Train_loss 0.4582683128780789. Val_loss 0.4691456917354039. Seconds 22.037943124771118
Epoch 10. Train_loss 0.45286133622366287. 

## 9. <a name="9">Test the classifier on the validation data</a>
(<a href="#0">Go to top</a>)

Let's get the validation predictions. Earlier we made predictions on the validation set with this line: ```model(data.to(device))```.

In [22]:
val_predictions = []
for data, target in val_loader:
    val_preds = model(data.to(device))
    val_predictions.extend(
        [np.rint(val_pred)[0] for val_pred in val_preds.detach().cpu().numpy()]
    )
print(val_predictions[:10])

[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0]


Confusion matrix, classification report and accuracy score are printed below.

In [23]:
# Use the fitted pipeline to make predictions on the validation dataset
print(confusion_matrix(val_label, val_predictions))
print(classification_report(val_label, val_predictions))
print("Accuracy (validation):", accuracy_score(val_label, val_predictions))

[[1327  725]
 [ 490 3058]]
              precision    recall  f1-score   support

         0.0       0.73      0.65      0.69      2052
         1.0       0.81      0.86      0.83      3548

    accuracy                           0.78      5600
   macro avg       0.77      0.75      0.76      5600
weighted avg       0.78      0.78      0.78      5600

Accuracy (validation): 0.7830357142857143


This score isn't an improvement over the single layer network from yesterday. RNNs usually require more data than regular neural networks in training. They also have additional hyperparameters to work with: __max_len, hidden_size, embed_size__ 

We will see some improved versions of RNNs namely [Gated Recurrent Units](https://pytorch.org/docs/1.9.1/generated/torch.nn.GRU.html) and [Long Sort-term Memory Networks](https://pytorch.org/docs/1.9.1/generated/torch.nn.LSTM.html) tomorrow. Those approaches will have a higher chance of success. 

## 10. <a name="10">Test the classifier on the unseen test data</a>
(<a href="#0">Go to top</a>)

Let's get the test predictions. 

In [24]:
df_test = pd.read_csv("../../data/examples/NLP-REVIEW-DATA-CLASSIFICATION-TEST.csv")

In [25]:
test_text = df_test["reviewText"].fillna(value="missing").tolist()

In [26]:
test_dataset = TensorDataset(pad_features(test_text, max_len)) #, torch.tensor(val_label))
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [27]:
test_predictions = []
for data, in test_loader:
    test_preds = model(data.to(device))
    test_predictions.extend(
        [np.rint(test_pred)[0] for test_pred in test_preds.detach().cpu().numpy()]
    )
print(test_predictions[:10])

[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]


In [28]:
import pandas as pd

result_df = pd.DataFrame()
result_df["ID"] = df_test["ID"]
result_df["isPositive"] = test_predictions

result_df.to_csv("result_day2_rnn.csv", encoding='utf-8', index=False)

## 11. <a name="11">Improvement ideas</a>
(<a href="#0">Go to top</a>)

We can improve our model by
* Changing hyper-parameters: Learning rate, batch size and hidden size
* Increase the number of layers: num_layers
* (Tomorrow) Switching to [Gated Recurrent Units](https://pytorch.org/docs/1.9.1/generated/torch.nn.GRU.html) and [Long Sort-term Memory Networks](https://pytorch.org/docs/1.9.1/generated/torch.nn.LSTM.html).